In [1]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

loader = TextLoader('./poker.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_documents(documents)


embeddings = HuggingFaceEmbeddings()

c:\Users\sathu\anaconda3\lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
Created a chunk of size 4257, which is longer than the specified 1000
Created a chunk of size 3099, which is longer than the specified 1000
C:\Users\sathu\AppData\Local\Temp\ipykernel_32404\3970803738.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings()
C:\Users\sathu\AppData\Local\Temp\ipykernel_32404\3970803738.py:12: LangChainDeprecationW

In [2]:
import os
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index_name = "langchain-demo"
checker = True
indexes = pc.list_indexes()
for index in indexes:
    if index_name == index.name:
       docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
       checker=False 
       break

if checker:    
    pc.create_index(name=index_name, metric="cosine", dimension=768, spec=ServerlessSpec(cloud='aws', region='us-east-1'))
    docsearch = Pinecone.from_documents(docs, embeddings, index_name=index_name)


MaxRetryError: HTTPSConnectionPool(host='api.pinecone.io', port=443): Max retries exceeded with url: /indexes (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001FE05468130>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

In [3]:
from langchain.llms import HuggingFaceHub

# Define the repo ID and connect to Mixtral model on Huggingface
repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
llm = HuggingFaceHub(
  repo_id=repo_id, 
  model_kwargs={"temperature": 0.8, "top_k": 50}, 
  huggingfacehub_api_token=os.getenv('HUGGING_FACE_API_KEY')
)

C:\Users\sathu\AppData\Local\Temp\ipykernel_37736\694083723.py:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  llm = HuggingFaceHub(


In [4]:
from langchain import PromptTemplate

template = """
You are a poker expert. These Human will ask you a questions about poker strategy. 
Use following piece of context to answer the question. 
If you don't know the answer, just say you don't know. 
Keep the answer within 2 sentences and concise. Use examples to explain where applicable.

Context: {context}
Question: {question}
Answer: 

"""

prompt = PromptTemplate(
  template=template, 
  input_variables=["context", "question"]
)

In [5]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": docsearch.as_retriever(),  "question": RunnablePassthrough()} 
  | prompt 
  | llm
  | StrOutputParser() 
)

In [6]:
class ChatBot():
  loader = TextLoader('./poker.txt')
  documents = loader.load()

  # The rest of the code here

  rag_chain = (
    {"context": docsearch.as_retriever(),  "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() 
  )

In [9]:
# Outside ChatBot() class
bot = ChatBot()
inp = input("Ask me anything: ")
result = bot.rag_chain.invoke(inp)
print(result)

MaxRetryError: HTTPSConnectionPool(host='langchain-demo-19t2r84.svc.aped-4627-b74a.pinecone.io', port=443): Max retries exceeded with url: /query (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001E900A1FC40>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))